In [1]:
import torch 

relation = torch.load('/home/tuannd/MBGCN/Tmall/item_buy.pth')
relation

/tmp/ipykernel_1437323/3699684860.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  relation = torch.load('/home/tuannd/MBGCN/Tmall/item_buy.pth')


tensor([[1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]], dtype=torch.int32)

In [2]:
relation.shape

torch.Size([11953, 11953])

In [5]:
relation.sum(dim=1).float().unsqueeze(-1)

tensor([[190.],
        [ 78.],
        [ 41.],
        ...,
        [ 42.],
        [ 38.],
        [ 28.]])

In [7]:
import torch  
import numpy as np  
from sklearn.metrics.pairwise import cosine_similarity  
import scipy.sparse as sp  
num_users = 41738
num_items = 11953

def create_item_similarity_matrix(behavior_file, num_items):  
    # Load user-item interactions  
    user_item_matrix = sp.lil_matrix((num_users, num_items))  
      
    with open(behavior_file, 'r') as f:  
        for line in f:  
            user, item = map(int, line.strip().split())  
            user_item_matrix[user, item] = 1.0  
      
    # Compute item-item similarity (transpose to get item-user matrix)  
    item_user_matrix = user_item_matrix.T.tocsr()  
      
    # Calculate cosine similarity between items  
    similarity_matrix = cosine_similarity(item_user_matrix)  
      
    # Convert to PyTorch tensor  
    similarity_tensor = torch.FloatTensor(similarity_matrix)  
      
    return similarity_tensor  
  
# Create similarity matrices for each behavior  
# behaviors = ['buy', 'cart', 'click', 'collect']  
behaviors = ['buy']
for behavior in behaviors:  
    similarity_matrix = create_item_similarity_matrix(f'Tmall/{behavior}.txt', num_items)  
    torch.save(similarity_matrix, f'item_{behavior}.pth')

In [8]:
relation_rebuild = torch.load('item_buy.pth')
print(relation_rebuild.shape)
relation_rebuild.sum(dim=1).float().unsqueeze(-1)

/tmp/ipykernel_1437323/1294529412.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  relation_rebuild = torch.load('item_buy.pth')


torch.Size([11953, 11953])


tensor([[9.7598],
        [4.5234],
        [4.4019],
        ...,
        [4.4411],
        [4.1706],
        [3.4723]])

In [11]:
# convert all > 0 value to 1
relation_rebuild = (relation_rebuild > 0).float()
print(relation_rebuild)

tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])


In [12]:
relation_rebuild.sum(dim=1).float().unsqueeze(-1)

tensor([[190.],
        [ 78.],
        [ 41.],
        ...,
        [ 42.],
        [ 38.],
        [ 28.]])